<h1 style="color: darkblue; font-family: 'Times New Roman', sans-serif;">
ECUACIÓN DE ESTADO DE PENG ROBINSON
</h1>

<h2 style="color: SlateBlue; font-family: 'Times New Roman', sans-serif;">
Calculo para mezclas multicomponentes
</h2>

In [1]:
import CoolProp.CoolProp as CP
import numpy as np
from scipy.optimize import fsolve
import requests
from PIL import Image
from io import BytesIO

In [2]:
# Universal gas constant (bar·m^3/mol·K)
R = 8.314462618 * 1e-5

# Ask the user how many substances they want to input
num_substances = int(input("Número de sustancias: \n"))

# Initialize a list to store information about each substance
substances = []

for i in range(num_substances):
    substance_name = str(input(f"Ingrese el nombre de la sustancia {i+1} en inglés: \n"))
    try: 
        # Get critical properties
        Pc = CP.PropsSI('PCRIT', substance_name) * 1e-5         # Critical pressure in bar
        Tc = CP.PropsSI('TCRIT', substance_name)                # Critical temperature in K
        w = CP.PropsSI('acentric', substance_name)              # Acentric factor (dimensionless)
        
        # Calculate 'a' and 'b' constants
        a = 0.45724 * (R * Tc)**2 / Pc                          # bar·m^6/mol^2
        b = 0.0778 * R * Tc / Pc                                # m^3/mol

        # Store the information in a dictionary
        substance_info = {
            "name": substance_name,
            "Pc": Pc,
            "Tc": Tc,
            "w": w,
            "a": a,
            "b": b
        }
        substances.append(substance_info)

    except Exception as e:
        print(f'Error: {e}')
        print(f'Sustancia {substance_name} no encontrada en la base de datos.')
        TC= float(input('Ingrese la temperatura critica de la sustancia en Kelvin : '))
        PC= float(input('Ingrese la Presion critica de la sustancia en bar : '))
        exit()
        
# Ask the user for the mole fractions
mole_fractions = []
for i in range(num_substances):
    x_i = float(input(f"Ingrese la fracción molar de la sustancia {substances[i]['name']}: \n"))
    mole_fractions.append(x_i)

# Calculate mix properties
Tpc  = sum(mole_fractions[i] * substances[i]['Tc'] for i in range(num_substances))
bmix = sum(mole_fractions[i] * substances[i]['b'] for i in range(num_substances))
wmix = sum(mole_fractions[i] * substances[i]['w'] for i in range(num_substances))
kmix = 0.37464 + 1.54226 * wmix - 0.26992 * wmix**2

# amix = sum(mole_fractions[i] * substances[i]['a'] for i in range(num_substances))
amix = 0
for i in range(num_substances):
    for j in range(num_substances):
        aij   = (substances[i]['a'] * substances[j]['a'])**0.5
        amix += mole_fractions[i] * mole_fractions[j] * aij

for substance in substances:
    print("\nInformación de la sustancia:")
    print(f"Nombre: {substance['name']}")
    print(f"Presión crítica: {substance['Pc']} bar")
    print(f"Temperatura crítica: {substance['Tc']} K\n")

print("Propiedades de la mezcla: ")
print(" amix: ", amix, "bar·m^6/mol^2")
print(" bmix: ", bmix, "m^3/mol")
print(" Temperatura pseudocrítica: ", Tpc, "K")
print(" kmix: ", kmix,"\n")
    
variable = str(input("Variable de estado (P, V o T): \n"))

if variable.lower() == "v":
    
    P = float(input("Presión (bar): "))
    T = float(input("Temperatura (K): "))
    Tr = T / Tpc
    alpha = (1 + kmix*(1 - Tr**0.5))**2
    
    def peng_robinson_v(V):
        term1 = R * T / (V - bmix)
        term2 = (amix * alpha) / (V * (V + bmix) + bmix * (V - bmix))
        return P - (term1 - term2)
    
    V_initial_guess = [bmix + 1e-5, R * T / P, 10 * bmix, 50 * bmix, 100 * bmix]
    
    V_m_roots = []
    for guess in V_initial_guess:
        try:
            root = fsolve(peng_robinson_v, guess)[0]
            if root > 0 and np.isreal(root):
                V_m_roots.append(root)
        except RuntimeError:
            # Handle cases where fsolve does not converge
            pass

    # Filter out duplicates
    V_m_roots = np.unique(np.round(np.array(V_m_roots),6))
    
    print("Volumen(es) en m^3/mol:")
    print(V_m_roots)
    resultados = [V_m_roots]

elif variable.lower() == "p":
    
    V = float(input("Volumen (m^3/mol): "))
    T = float(input("Temperatura (K): "))
    Tr = T / Tpc
    alpha = (1 + kmix*(1 - Tr**0.5))**2
    
    P = R*T/ (V - bmix) + amix*alpha / (V*(V+bmix) + bmix*(V-bmix))
    
    print(P, "bar")
    resultados = P
    
elif variable.lower() == "t":
    
    V = float(input("Volumen (m^3/mol): "))
    P = float(input("Presión (bar): "))
    
    
    def peng_robinson_t(T):
        term1 = R * T / (V - bmix)
        term2 = (amix * (1 + kmix*(1 - (T/Tpc)**0.5)**2)) / (V * (V + bmix) + bmix * (V - bmix))
        return P - (term1 - term2)
    
    
    
    T_initial_guess = [1/2 * Tpc, Tpc, 2 * Tpc, 3 * Tpc, 4 * Tpc]
    
    T_m_roots = []
    for guess in T_initial_guess:
        try:
            root = fsolve(peng_robinson_t, guess)[0]
            if root > 0 and np.isreal(root):
                T_m_roots.append(root)
        except RuntimeError:
            # Handle cases where fsolve does not converge
            pass

    # Filter out duplicates
    T_m_roots = np.unique(np.round(np.array(T_m_roots),6))
    
    print("Temperatura(s) en K: ")
    print(T_m_roots)

    resultados = [T_m_roots]
 
else: 
    print("Variable inválida")

Número de sustancias: 
 3
Ingrese el nombre de la sustancia 1 en inglés: 
 Hydrogen
Ingrese el nombre de la sustancia 2 en inglés: 
 Water
Ingrese el nombre de la sustancia 3 en inglés: 
 Ethane
Ingrese la fracción molar de la sustancia Hydrogen: 
 0.5
Ingrese la fracción molar de la sustancia Water: 
 0.4
Ingrese la fracción molar de la sustancia Ethane: 
 0.1



Información de la sustancia:
Nombre: Hydrogen
Presión crítica: 12.964 bar
Temperatura crítica: 33.145 K


Información de la sustancia:
Nombre: Water
Presión crítica: 220.64000000000001 bar
Temperatura crítica: 647.096 K


Información de la sustancia:
Nombre: Ethane
Presión crítica: 48.722 bar
Temperatura crítica: 305.322 K

Propiedades de la mezcla: 
 amix:  2.2034447941419204e-06 bar·m^6/mol^2
 bmix:  1.9911381620123156e-05 m^3/mol
 Temperatura pseudocrítica:  305.9431 K
 kmix:  0.43303390309098194 



Variable de estado (P, V o T): 
 p
Volumen (m^3/mol):  0.00125
Temperatura (K):  300


21.65629736559232 bar


In [14]:
from IPython.display import display, Markdown
from dotenv import load_dotenv
import os
import openai

#Asignar la clave API y crear cliente para chatty
load_dotenv("xxx.env")

# Recupera la API key
api_key = os.getenv("OPENAI_API_KEY")
if api_key :
    print("Acceso permitido")
else:
    print("No se encontró la clave")

client = openai.OpenAI(api_key=api_key)

Acceso permitido


In [15]:

response = client.chat.completions.create(
    model ="gpt-4o",
    messages =[
        {"role": "system", "content": "Eres un asitente util, sabes de termodinámica y ecuaciónes de estado, vas directamente a lo que el usuario te pide, eres claro y conciso, evita el uso de codigo latex"},
        {"role": "user", "content": f'''
        ECUACIÓN DE ESTADO DE PENG ROBINSON
        Las sustancias son las siguientes: {[substances]}
        Propiedades de la mezcla:
        amix:  {amix} bar·m^6/mol^2
        bmix: {bmix} m^3/mol
        Temperatura pseudocrítica: {Tpc} K
        kmix: {kmix}"\n")
        Donde la mezcla esta en las siguientes condiciones:
        Temperatura: {T} kelvin
        Volumen especifico: {V} m3/mol
        Se obtuvieron los siguientes resultados: {resultados}
        Haz un resumen de los datos obtenidos y
        ¿Que puedes concluir de esta respuesta? ¿Es valida y tiene sentido fisico?
        '''}
    ],
    #Establecer limite de tokens
    #max_tokens=50 
)

respuesta = (response.choices[0].message.content)

display(Markdown(respuesta))

**Resumen de los datos obtenidos:**

1. **Componentes de la mezcla:**
   - Hidrógeno: 
     - Pc: 12.964 bar
     - Tc: 33.145 K
     - w: -0.219
     - a: 2.679e-07 bar·m^6/mol^2
     - b: 1.654e-05 m^3/mol
   - Agua:
     - Pc: 220.64 bar
     - Tc: 647.096 K
     - w: 0.344
     - a: 5.999e-06 bar·m^6/mol^2
     - b: 1.897e-05 m^3/mol
   - Etano:
     - Pc: 48.722 bar
     - Tc: 305.322 K
     - w: 0.099
     - a: 6.048e-06 bar·m^6/mol^2
     - b: 4.054e-05 m^3/mol

2. **Propiedades de la mezcla:**
   - amix: 2.203e-06 bar·m^6/mol^2
   - bmix: 1.991e-05 m^3/mol
   - Temperatura pseudocrítica: 305.9431 K
   - kmix: 0.4330

3. **Condiciones de la mezcla:**
   - Temperatura: 300.0 K
   - Volumen específico: 0.00125 m^3/mol

4. **Resultado obtenido:** 21.656 bar 

**Conclusión sobre la validez y sentido físico:**

El resultado de 21.656 bar se refiere a la presión de la mezcla calculada usando la ecuación de estado de Peng-Robinson en las condiciones especificadas. Para que el resultado tenga sentido físico, debe coincidir con expectativas razonables basadas en los componentes y condiciones. Aquí están algunos puntos a considerar:

- La mezcla incluye agua, que tiene un punto crítico alto, en combinación con hidrógeno, que tiene un punto crítico bajo. Esto sugiere que el comportamiento de la mezcla puede ser complejo.
  
- La presión calculada (21.656 bar) parece elevada pero puede ser razonable dado que las condiciones pseudocríticas de la mezcla y las contribuciones de cada componente se han considerado adecuadamente en la ecuación de Peng-Robinson.

- Asegúrate de que las constantes 'a' y 'b' para los componentes y la mezcla han sido calculadas correctamente. Cualquier error en estos parámetros afectará la precisión del resultado.

Por lo tanto, el resultado parece razonable, siempre que todos los cálculos necesarios para 'a', 'b', y propiedades de mezcla se realicen correctamente. Validar el resultado con datos experimentales, si están disponibles, sería ideal para una mayor confirmación.